**Data Preprocessing**

We need to check the images in our dataset for their sizes and number of color components

In [1]:
""" Import necessary libs"""

import pandas as pd
import numpy as np
import cv2 as cv
import os
import glob


In [3]:
""" Check image size and color components"""
df = pd.DataFrame()

targetdir = "images"
filelist = glob.glob(targetdir+str("/*"))
for file in filelist:
    img = cv.imread(file)
    img_shape = img.shape
    #print(img_shape)
    df = df.append(pd.Series(img_shape),ignore_index=True)
df = df.rename(columns={0: "Width", 1: "Height", 2:"Components"})
df.head()

,Width,Height,Components
0,350.0,350.0,3.0
1,350.0,350.0,3.0
2,350.0,350.0,3.0
3,350.0,350.0,3.0
4,350.0,350.0,3.0


**Now we need to determine the minimum & maximum width and height from all the image sizes**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13718 entries, 0 to 13717
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Width       13718 non-null  float64
 1   Height      13718 non-null  float64
 2   Components  13718 non-null  float64
dtypes: float64(3)
memory usage: 321.6 KB


In [5]:
df =df.rename(columns={"Rows": "Width", "Columns": "Height", 2:"Components"})
df.agg(['max', 'min', 'mean', 'std'])

,Width,Height,Components
max,536.000000,441.000000,3.0
min,24.000000,18.000000,3.0
mean,335.768698,333.548039,3.0
std,58.700772,63.898948,0.0


In [6]:
# Load in image data
image_info = pd.read_csv('data/legend.csv',delimiter=',')
image_info.head()
image_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13690 entries, 0 to 13689
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user.id  13690 non-null  object
 1   image    13690 non-null  object
 2   emotion  13690 non-null  object
dtypes: object(3)
memory usage: 321.0+ KB


In [7]:
image_info.head()

,user.id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
1,628,facial-expressions_2868585k.jpg,surprise
2,628,facial-expressions_2868584k.jpg,disgust
3,628,facial-expressions_2868582k.jpg,fear
4,dwdii,Aaron_Eckhart_0001.jpg,neutral


In [8]:
image_info["emotion"].unique()

array(['anger', 'surprise', 'disgust', 'fear', 'neutral', 'happiness',
       'sadness', 'contempt', 'NEUTRAL', 'SADNESS', 'DISGUST', 'FEAR',
       'SURPRISE', 'ANGER', 'HAPPINESS'], dtype=object)

In [9]:
image_info["emotion"].value_counts()

neutral      6717
happiness    5309
HAPPINESS     387
surprise      356
anger         228
DISGUST       195
NEUTRAL       151
SADNESS       144
sadness       124
ANGER          24
fear           13
disgust        13
SURPRISE       12
contempt        9
FEAR            8
Name: emotion, dtype: int64

In [10]:
# converting the emotion string to lowercase
image_info["emotion"] = image_info["emotion"].str.lower()
image_info["emotion"].value_counts()

neutral      6868
happiness    5696
surprise      368
sadness       268
anger         252
disgust       208
fear           21
contempt        9
Name: emotion, dtype: int64

It can be easily noticable that the emotion categories are not equally distributed here. Hence we need to drop some images from neutral and happiness category to make the distribution even for all categories.

In [11]:
""" Merge the contempt into disgust category"""
image_info["emotion"] = image_info["emotion"].replace("contempt","disgust")

In [12]:
image_info["emotion"].value_counts()

neutral      6868
happiness    5696
surprise      368
sadness       268
anger         252
disgust       217
fear           21
Name: emotion, dtype: int64

In [13]:
#shuffle the dataset , iterate over it and select 500 images from neutral and happiness category
image_info = image_info.sample(frac=1).reset_index(drop=True)

In [18]:
#if shuffle is performed again remove already selected images and perform the partition using below cell again
#!rm -f selectedImages/*

In [23]:
#move above selected images into a separate dataset folder
neutral = 0
happy = 0
selectedImages = pd.DataFrame(columns=["image","emotion"])
for index, row in image_info.iterrows():
    #if happy >= 500 and neutral >=500:
     #   break
    if row["emotion"] == "happiness":
        happy += 1
        if happy > 500:
            continue
    elif row["emotion"] == "neutral":
        neutral += 1
        if neutral > 500:
            continue
    selectedImages = selectedImages.append(pd.Series(row),ignore_index=True)
    #print(row)
    
    ##uncomment if need to copy the images to another folder
    
    #cmdstring = "cp images/{} selectedImages/.".format(row["image"])
    #os.system(cmdstring)

In [24]:

selectedImages["emotion"].value_counts()

neutral      500
happiness    500
surprise     368
sadness      268
anger        252
disgust      217
fear          21
Name: emotion, dtype: int64

In [25]:
!ls -l selectedImages/ | wc -l

2126


In [26]:

selectedImages = selectedImages.drop(['user.id'], axis=1) #we donot need user.id

In [27]:
selectedImages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    2126 non-null   object
 1   emotion  2126 non-null   object
dtypes: object(2)
memory usage: 33.3+ KB


In [28]:
#divide this dataset into test train and validation set
train, validate, test = np.split(selectedImages.sample(frac=1), [int(.6*len(selectedImages)), int(.8*len(selectedImages))])

#https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1275 entries, 41 to 1714
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1275 non-null   object
 1   emotion  1275 non-null   object
dtypes: object(2)
memory usage: 29.9+ KB


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 514 to 474
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    426 non-null    object
 1   emotion  426 non-null    object
dtypes: object(2)
memory usage: 10.0+ KB


In [32]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 492 to 1955
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    425 non-null    object
 1   emotion  425 non-null    object
dtypes: object(2)
memory usage: 10.0+ KB


In [33]:
train["emotion"].value_counts()

neutral      305
happiness    294
surprise     217
sadness      163
anger        155
disgust      127
fear          14
Name: emotion, dtype: int64

In [34]:
validate["emotion"].value_counts()

happiness    107
neutral      104
surprise      68
sadness       53
disgust       47
anger         45
fear           1
Name: emotion, dtype: int64

In [35]:
test["emotion"].value_counts()

happiness    99
neutral      91
surprise     83
anger        52
sadness      52
disgust      43
fear          6
Name: emotion, dtype: int64

In [50]:
def categorical2Numerical(df):
    category = {'happiness': 0, 'neutral': 1, 'surprise':2, 'anger':3, 'sadness': 4, 'disgust':5, 'fear':6}
    df.emotion = [category[item] for item in df.emotion]
    return df
train["emotion"].value_counts()

1    305
0    294
2    217
4    163
3    155
5    127
6     14
Name: emotion, dtype: int64

In [55]:
#saving category mapping so when we need to reverse map the value to it's original emotion 
import json

with open('category.json', 'w') as fp:
    json.dump(category, fp)

In [51]:

validate = categorical2Numerical(validate)
test = categorical2Numerical(test)


In [52]:
validate["emotion"].value_counts()

0    107
1    104
2     68
4     53
5     47
3     45
6      1
Name: emotion, dtype: int64

In [53]:
test["emotion"].value_counts()

0    99
1    91
2    83
4    52
3    52
5    43
6     6
Name: emotion, dtype: int64

In [67]:

def dropColumn(df,i):
    df = df.drop(df.columns[i], axis=1)
    return df

In [68]:
train= dropColumn(train,0)
train.head()

,emotion
41,1
381,1
107,0
1172,4
1912,2


From above the distribution looks justified for each category of emotions.

In [54]:
#we need to save this selected images data frame into a csv files so that we can load them again as they are.
selectedImages.to_csv('selectedImages.csv')
train.to_csv('train.csv')
validate.to_csv('validate.csv')
test.to_csv('test.csv')

In [38]:
def moveImageToFolder(df, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for index, row in df.iterrows():
        cmdstring = "mv selectedImages/{} {}/.".format(row["image"],folder)
        os.system(cmdstring)

In [39]:
moveImageToFolder(train,"selectedImages/train")

In [40]:
moveImageToFolder(validate,"selectedImages/valid")
moveImageToFolder(test,"selectedImages/test")

In [95]:
def createCategoryFolder(df, folder):
    category = {0:'happiness', 1:'neutral', 2:'surprise', 3:'anger', 4:'sadness', 5:'disgust', 6:'fear'}
    for index, row in df.iterrows():
        
        currentPath = os.path.join(folder,category[row["emotion"]])
        if not os.path.exists(currentPath):
            os.makedirs(currentPath)
        cmdstring = "mv {}/{} {}/.".format(folder,row["image"],currentPath)
        print(cmdstring)
        os.system(cmdstring)

In [88]:
train = pd.read_csv('selectedImages/train.csv',delimiter=',')


In [98]:
createCategoryFolder(train,"selectedImages/train")

mv selectedImages/train/George_W_Bush_0257.jpg selectedImages/train/neutral/.
mv selectedImages/train/Dan_Wheldon_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Emmy_Rossum_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Avinash_18.jpg selectedImages/train/sadness/.
mv selectedImages/train/Oprah_Winfrey_0002.jpg selectedImages/train/surprise/.
mv selectedImages/train/Bobby_Robson_0002.jpg selectedImages/train/neutral/.
mv selectedImages/train/Bob_Huggins_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/Dwarakish_107.jpg selectedImages/train/disgust/.
mv selectedImages/train/Jim_Furyk_0006.jpg selectedImages/train/neutral/.
mv selectedImages/train/FaridaJalal_75.jpg selectedImages/train/sadness/.
mv selectedImages/train/FaridaJalal_57.jpg selectedImages/train/disgust/.
mv selectedImages/train/Lee_Hoi-chang_0003.jpg selectedImages/train/surprise/.
mv selectedImages/train/Dwarakish_33.jpg selectedImages/train/disgust/.
mv selectedImages/trai

mv selectedImages/train/Mike_Krzyzewski_0005.jpg selectedImages/train/anger/.
mv selectedImages/train/Jennifer_Capriati_0002.jpg selectedImages/train/anger/.
mv selectedImages/train/Kevin_Spacey_0004.jpg selectedImages/train/happiness/.
mv selectedImages/train/Eduard_Shevardnadze_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Tim_Lobinger_0001.jpg selectedImages/train/sadness/.
mv selectedImages/train/Ismail_Cem_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Silvio_Berlusconi_0010.jpg selectedImages/train/neutral/.
mv selectedImages/train/Tony_Shalhoub_0004.jpg selectedImages/train/neutral/.
mv selectedImages/train/Stan_Heath_0002.jpg selectedImages/train/surprise/.
mv selectedImages/train/Rick_Carlisle_0002.jpg selectedImages/train/anger/.
mv selectedImages/train/HrithikRoshan_6.jpg selectedImages/train/sadness/.
mv selectedImages/train/Al_Davis_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Lindsay_Davenport_0013.jpg selectedImag

mv selectedImages/train/Dileep_105.jpg selectedImages/train/sadness/.
mv selectedImages/train/Keanu_Reeves_0011.jpg selectedImages/train/neutral/.
mv selectedImages/train/Fernando_Gonzalez_0004.jpg selectedImages/train/surprise/.
mv selectedImages/train/Kofi_Annan_0029.jpg selectedImages/train/neutral/.
mv selectedImages/train/Jeane_Kirkpatrick_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/George_Gregan_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/Dwarakish_9.jpg selectedImages/train/fear/.
mv selectedImages/train/Yoon_Jeong_Cho_0001.jpg selectedImages/train/sadness/.
mv selectedImages/train/George_W_Bush_0443.jpg selectedImages/train/neutral/.
mv selectedImages/train/Robert_Redford_0002.jpg selectedImages/train/neutral/.
mv selectedImages/train/Steven_Feldman_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/Li_Peng_0005.jpg selectedImages/train/happiness/.
mv selectedImages/train/Charlie_Sheen_0001.jpg selectedImages/train/neutral

mv selectedImages/train/Peter_Arnett_0003.jpg selectedImages/train/neutral/.
mv selectedImages/train/Jessica_Lange_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Josh_Evans_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/Darko_Milicic_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/Jean_Chretien_0010.jpg selectedImages/train/anger/.
mv selectedImages/train/Martin_Scorsese_0004.jpg selectedImages/train/happiness/.
mv selectedImages/train/Vladimir_Spidla_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/George_W_Bush_0115.jpg selectedImages/train/neutral/.
mv selectedImages/train/FaridaJalal_36.jpg selectedImages/train/neutral/.
mv selectedImages/train/Martha_Lucia_Ramirez_0004.jpg selectedImages/train/surprise/.
mv selectedImages/train/Mariana_Pollack_0002.jpg selectedImages/train/neutral/.
mv selectedImages/train/Thomas_Enqvist_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/Ariel_Sharon_0035.jpg selectedImag

mv selectedImages/train/Lope_Mendoza_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Dileep_40.jpg selectedImages/train/neutral/.
mv selectedImages/train/Lenny_Wilkens_0002.jpg selectedImages/train/surprise/.
mv selectedImages/train/George_W_Bush_0296.jpg selectedImages/train/happiness/.
mv selectedImages/train/Albrecht_Mentz_0002.jpg selectedImages/train/neutral/.
mv selectedImages/train/HrithikRoshan_31.jpg selectedImages/train/sadness/.
mv selectedImages/train/Jennifer_Capriati_0009.jpg selectedImages/train/anger/.
mv selectedImages/train/Paul_Burrell_0011.jpg selectedImages/train/happiness/.
mv selectedImages/train/Colin_Cowie_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Claudia_Schiffer_0002.jpg selectedImages/train/happiness/.
mv selectedImages/train/Kenneth_Dam_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Mike_Tyson_0002.jpg selectedImages/train/happiness/.
mv selectedImages/train/Spike_Helmick_0001.jpg selectedImages/train

...

mv selectedImages/train/Roberto_Laratro_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/Kieran_Prendergast_0002.jpg selectedImages/train/neutral/.
mv selectedImages/train/Brian_Heidik_0002.jpg selectedImages/train/happiness/.
mv selectedImages/train/John_Eder_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Chris_Moore_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Alvaro_Noboa_0002.jpg selectedImages/train/anger/.
mv selectedImages/train/facial-expressions_2868585k.jpg selectedImages/train/surprise/.
mv selectedImages/train/Jayamadhuri_179.jpg selectedImages/train/sadness/.
mv selectedImages/train/Elijan_Ingram_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/John_Kerry_0015.jpg selectedImages/train/surprise/.
mv selectedImages/train/Serena_Williams_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/Hamid_Karzai_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Angelina_Jolie_0005.jpg selected

mv selectedImages/train/Rowan_Williams_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Eileen_Coparropa_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Jeremy_Greenstock_0014.jpg selectedImages/train/surprise/.
mv selectedImages/train/Dwarakish_35.jpg selectedImages/train/disgust/.
mv selectedImages/train/Mamdouh_Habib_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Avinash_8.jpg selectedImages/train/sadness/.
mv selectedImages/train/Luther_Htu_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Hamid_Karzai_0017.jpg selectedImages/train/neutral/.
mv selectedImages/train/Megawati_Sukarnoputri_0010.jpg selectedImages/train/happiness/.
mv selectedImages/train/Vladimir_Putin_0017.jpg selectedImages/train/neutral/.
mv selectedImages/train/Charles_Moose_0006.jpg selectedImages/train/anger/.
mv selectedImages/train/Robert_Horan_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Carlos_Vives_0001.jpg selectedImages/tra

mv selectedImages/train/HrithikRoshan_8.jpg selectedImages/train/happiness/.
mv selectedImages/train/John_Kerry_0002.jpg selectedImages/train/surprise/.
mv selectedImages/train/Rose_Marie_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/James_Blake_0014.jpg selectedImages/train/surprise/.
mv selectedImages/train/Dwarakish_175.jpg selectedImages/train/disgust/.
mv selectedImages/train/Rudolph_Giuliani_0004.jpg selectedImages/train/surprise/.
mv selectedImages/train/Larry_Hahn_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/George_Blaney_0001.jpg selectedImages/train/anger/.
mv selectedImages/train/Martha_Martinez_Flores_0001.jpg selectedImages/train/sadness/.
mv selectedImages/train/Garry_Trudeau_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Jennifer_Capriati_0039.jpg selectedImages/train/anger/.
mv selectedImages/train/Carolina_Kluft_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Bill_Simon_0002.jpg selectedImages/trai

mv selectedImages/train/Dwarakish_102.jpg selectedImages/train/sadness/.
mv selectedImages/train/Thomas_Ferguson_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Jackie_Chan_0006.jpg selectedImages/train/fear/.
mv selectedImages/train/Annie-Jeanne_Reynaud_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/FaridaJalal_102.jpg selectedImages/train/disgust/.
mv selectedImages/train/Dwarakish_2.jpg selectedImages/train/sadness/.
mv selectedImages/train/Hashim_Thaci_0002.jpg selectedImages/train/happiness/.
mv selectedImages/train/Brooke_Gordon_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Jack_Straw_0027.jpg selectedImages/train/surprise/.
mv selectedImages/train/Pervez_Musharraf_0016.jpg selectedImages/train/happiness/.
mv selectedImages/train/Choi_Yun-yong_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Gustavo_Kuerten_0002.jpg selectedImages/train/sadness/.
mv selectedImages/train/Kim_Clijsters_0008.jpg selectedImages

mv selectedImages/train/Harrison_Ford_0006.jpg selectedImages/train/neutral/.
mv selectedImages/train/Clara_Harris_0001.jpg selectedImages/train/sadness/.
mv selectedImages/train/Jayamadhuri_95.jpg selectedImages/train/sadness/.
mv selectedImages/train/Colin_Powell_0224.jpg selectedImages/train/sadness/.
mv selectedImages/train/Dwarakish_3.jpg selectedImages/train/disgust/.
mv selectedImages/train/Amelie_Mauresmo_0010.jpg selectedImages/train/happiness/.
mv selectedImages/train/Richard_Myers_0009.jpg selectedImages/train/neutral/.
mv selectedImages/train/William_Ragland_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Lisa_Ling_0001.jpg selectedImages/train/happiness/.
mv selectedImages/train/Ian_Huntley_0001.jpg selectedImages/train/neutral/.
mv selectedImages/train/Maxim_Afinogenov_0001.jpg selectedImages/train/surprise/.
mv selectedImages/train/Mike_Weir_0003.jpg selectedImages/train/surprise/.
mv selectedImages/train/George_W_Bush_0130.jpg selectedImages/train/happin

In [96]:
createCategoryFolder(test,"selectedImages/test")

mv selectedImages/test/Tony_Blair_0027.jpg selectedImages/test/happiness/.
mv selectedImages/test/Lenny_Wilkens_0001.jpg selectedImages/test/surprise/.
mv selectedImages/test/Dwarakish_154.jpg selectedImages/test/sadness/.
mv selectedImages/test/Paradorn_Srichaphan_0003.jpg selectedImages/test/neutral/.
mv selectedImages/test/George_W_Bush_0301.jpg selectedImages/test/neutral/.
mv selectedImages/test/Kim_Clijsters_0012.jpg selectedImages/test/disgust/.
mv selectedImages/test/Tyler_Hamilton_0001.jpg selectedImages/test/disgust/.
mv selectedImages/test/HrithikRoshan_115.jpg selectedImages/test/disgust/.
mv selectedImages/test/Claire_Leger_0001.jpg selectedImages/test/sadness/.
mv selectedImages/test/Jennifer_Aniston_0021.jpg selectedImages/test/surprise/.
mv selectedImages/test/FaridaJalal_11.jpg selectedImages/test/happiness/.
mv selectedImages/test/John_Kerry_0003.jpg selectedImages/test/surprise/.
mv selectedImages/test/Jayamadhuri_133.jpg selectedImages/test/disgust/.
mv selectedImag

mv selectedImages/test/Peter_Hunt_0001.jpg selectedImages/test/happiness/.
mv selectedImages/test/Dwarakish_28.jpg selectedImages/test/fear/.
mv selectedImages/test/George_W_Bush_0184.jpg selectedImages/test/neutral/.
mv selectedImages/test/Ana_Guevara_0006.jpg selectedImages/test/happiness/.
mv selectedImages/test/Jayamadhuri_153.jpg selectedImages/test/disgust/.
mv selectedImages/test/Jayamadhuri_122.jpg selectedImages/test/disgust/.
mv selectedImages/test/Helen_Darling_0001.jpg selectedImages/test/surprise/.
mv selectedImages/test/Dwarakish_183.jpg selectedImages/test/disgust/.
mv selectedImages/test/Colin_Powell_0204.jpg selectedImages/test/neutral/.
mv selectedImages/test/Amelie_Mauresmo_0019.jpg selectedImages/test/neutral/.
mv selectedImages/test/Maurice_Strong_0001.jpg selectedImages/test/happiness/.
mv selectedImages/test/Joan_Claybrook_0001.jpg selectedImages/test/surprise/.
mv selectedImages/test/John_Swofford_0002.jpg selectedImages/test/sadness/.
mv selectedImages/test/Cha

mv selectedImages/test/Jennifer_Capriati_0020.jpg selectedImages/test/anger/.
mv selectedImages/test/KatrinaKaif_92.jpg selectedImages/test/sadness/.
mv selectedImages/test/Jack_Straw_0004.jpg selectedImages/test/anger/.
mv selectedImages/test/Tyler_Hamilton_0002.jpg selectedImages/test/happiness/.
mv selectedImages/test/Janela_Jara_0001.jpg selectedImages/test/sadness/.
mv selectedImages/test/Catherine_Ndereba_0001.jpg selectedImages/test/neutral/.
mv selectedImages/test/Raoul_Ruiz_0001.jpg selectedImages/test/happiness/.
mv selectedImages/test/Martin_ONeill_0001.jpg selectedImages/test/happiness/.
mv selectedImages/test/Glenn_Plummer_0001.jpg selectedImages/test/neutral/.
mv selectedImages/test/Dwarakish_111.jpg selectedImages/test/disgust/.
mv selectedImages/test/Orrin_Hatch_0002.jpg selectedImages/test/surprise/.
mv selectedImages/test/Alvaro_Uribe_0031.jpg selectedImages/test/happiness/.
mv selectedImages/test/Dwarakish_131.jpg selectedImages/test/sadness/.
mv selectedImages/test/

mv selectedImages/test/Serena_Williams_0015.jpg selectedImages/test/anger/.
mv selectedImages/test/Amber_Frey_0001.jpg selectedImages/test/sadness/.
mv selectedImages/test/Casy_Preslar_0001.jpg selectedImages/test/surprise/.
mv selectedImages/test/FaridaJalal_105.jpg selectedImages/test/disgust/.
mv selectedImages/test/Mahathir_Mohamad_0003.jpg selectedImages/test/disgust/.
mv selectedImages/test/Meles_Zenawi_0001.jpg selectedImages/test/neutral/.
mv selectedImages/test/Ray_Sherman_0001.jpg selectedImages/test/happiness/.
mv selectedImages/test/Rick_Stansbury_0002.jpg selectedImages/test/surprise/.
mv selectedImages/test/HrithikRoshan_148.jpg selectedImages/test/surprise/.
mv selectedImages/test/Cheryl_Tiegs_0001.jpg selectedImages/test/neutral/.
mv selectedImages/test/Chris_Neil_0001.jpg selectedImages/test/surprise/.
mv selectedImages/test/Sarah_Hughes_0006.jpg selectedImages/test/surprise/.
mv selectedImages/test/Lance_Bass_0002.jpg selectedImages/test/happiness/.
mv selectedImages/

In [97]:
createCategoryFolder(validate,"selectedImages/valid")

mv selectedImages/valid/Wan_Yanhai_0001.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/HrithikRoshan_44.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Hannah_Stockbauer_0002.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Alexander_Losyukov_0003.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Bruce_Springsteen_0003.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/Wilfredo_Moreno_0001.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Brian_Scalabrine_0001.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Doc_Rivers_0001.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Dan_Prinster_0001.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Chang_Dae-whan_0002.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Dileep_85.jpg selectedImages/valid/anger/.
mv selectedImages/valid/George_W_Bush_0368.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Mike_Montgomery_0002.jpg selectedImages/va

mv selectedImages/valid/Jayamadhuri_74.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Jason_Alexander_0001.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Matthew_Broderick_0003.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/SharmilaTagore_43.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Jayamadhuri_195.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/HrithikRoshan_120.jpg selectedImages/valid/disgust/.
mv selectedImages/valid/Arnold_Schwarzenegger_0024.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/Tiger_Woods_0001.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Geno_Auriemma_0001.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Tom_Jones_0001.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/Norah_Jones_0010.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/HrithikRoshan_180.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/James_McGreevey_0001.jpg selectedImages/v

mv selectedImages/valid/Recep_Tayyip_Erdogan_0018.jpg selectedImages/valid/anger/.
mv selectedImages/valid/Jiang_Zemin_0018.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Lindsay_Davenport_0012.jpg selectedImages/valid/anger/.
mv selectedImages/valid/George_W_Bush_0297.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Thomas_Scavone_0001.jpg selectedImages/valid/disgust/.
mv selectedImages/valid/Vanessa_Williams_0002.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/HrithikRoshan_192.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/James_Parker_0002.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/Jose_Theodore_0001.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Pascal_Quignard_0003.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Donald_Rumsfeld_0065.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Dwarakish_58.jpg selectedImages/valid/disgust/.
mv selectedImages/valid/George_W_Bush_0356.jpg sel

mv selectedImages/valid/Andre_Agassi_0014.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Robert_Schuller_0001.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Tom_Curley_0001.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Paradorn_Srichaphan_0002.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/James_Carville_0001.jpg selectedImages/valid/anger/.
mv selectedImages/valid/SharmilaTagore_37.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Prince_Naruhito_0001.jpg selectedImages/valid/happiness/.
mv selectedImages/valid/Dwarakish_19.jpg selectedImages/valid/disgust/.
mv selectedImages/valid/Luke_Smith_0001.jpg selectedImages/valid/neutral/.
mv selectedImages/valid/Julie_Gerberding_0015.jpg selectedImages/valid/surprise/.
mv selectedImages/valid/FaridaJalal_74.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Dileep_83.jpg selectedImages/valid/sadness/.
mv selectedImages/valid/Agnelo_Queiroz_0001.jpg selectedImages/valid/ne

In [1]:
import boto3
import sagemaker

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [3]:

# should be the name of directory you created to save your features data
data_dir = 'selectedImages'

# set prefix, a descriptive name for a directory  
prefix = 'emotion-detection'

# upload all data to S3
#input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [4]:
# confirm that data is in S3 bucket

empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)
    break
print(len(empty_check))
assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

emotion-detection/output/sagemaker-pytorch-2020-12-04-17-51-14-952/debug-output/training_job_end.ts
1
Test passed!


In [4]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [18]:
train_dir = './' + data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [19]:
import torch
import torch.utils.data
import pandas as pd
import os

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), names=None, nrows=50)

# Turn the input pandas dataframe into tensors
#print(train_sample.head())
#train_sample_y = torch.from_numpy(train_sample['emotion'].values)
train_transforms = transforms.Compose([transforms.Grayscale(1),
                                       transforms.RandomRotation(30, fill=(0,)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, ), (0.5, ))])
valid_transforms = transforms.Compose([transforms.Grayscale(1),
                                       transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, ), (0.5, ))])
test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                             std=[0.229, 0.224, 0.225])])
# Build the dataset

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
# TODO: Using the image datasets and the trainforms, define the dataloaders


trainloader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=10, shuffle=False)
testloader = torch.utils.data.DataLoader(test_data, batch_size=10)

In [65]:
#images_test, labels_test = next(iter(trainloader))
#images_test.shape
#labels_test.shape
train_data.class_to_idx



{'anger': 0,
 'disgust': 1,
 'fear': 2,
 'happiness': 3,
 'neutral': 4,
 'sadness': 5,
 'surprise': 6}

In [9]:
#!pygmentize model.py

In [10]:
#!pygmentize train.py

In [55]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout

class EmotionClassifier(nn.Module):
    """
    This is the simple CNN model we will be using to perform emotion classification for seven emotions
    """

    def __init__(self):
        """
        Initialize the model by settingg up the various layers.
        """
        super(EmotionClassifier, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
        )

        self.linear_layers = nn.Sequential(
            Linear(128 * 14 * 14, 512),
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,84),
            ReLU(inplace=True),
            Dropout(),
            Linear(84,7)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


In [70]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    prev_acc = 0
    accuracy = 0
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
        #for batch_idx, (data, target) in enumerate(train_loader):
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            optimizer.zero_grad()

            logps = model.forward(batch_X)
            m = nn.LogSoftmax(dim=1)
            #nll_loss = nn.NLLLoss()
            #output = nll_loss(m(input), target)
            loss = loss_fn(m(logps), batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.data.item()
        print("Epoch: {}, NLLLoss: {}".format(epoch, total_loss / len(train_loader)))
        model.eval()
        valid_loss = 0
        #accuracy = 0
        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model.forward(inputs)
                m = nn.LogSoftmax(dim=1)
                batch_loss = loss_fn(m(logps), labels)

                valid_loss += batch_loss.item()

                # Calculate accuracy
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                valid_acc = accuracy/len(validloader)

        print(f"Epoch {epoch+1}/{epochs}.. "
      f"Train loss: {total_loss / len(train_loader):.3f}.. "
      f"Valid loss: {valid_loss/len(validloader):.3f}.. "
      f"Validation accuracy: {valid_acc:.3f}")
        
        model.train()
        if valid_acc-prev_acc < .1 and valid_acc > .9:
            break
        else:
            prev_acc = valid_acc

In [21]:
import torch.optim as optim
#import EmotionClassifier as E
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = EmotionClassifier().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.NLLLoss() #torch.nn.BCELoss()

#train(model, trainloader, 2, optimizer, loss_fn, device)

In [86]:
!pip install sagemaker-containers

     |████████████████████████████████| 51 kB 333 kB/s eta 0:00:011
     |████████████████████████████████| 77 kB 5.9 MB/s  eta 0:00:01
  Created wheel for sagemaker-containers: filename=sagemaker_containers-2.8.6.post2-cp36-cp36m-linux_x86_64.whl size=68819 sha256=d017d44b3761dac2c438d71d091bcb77b03b556090ae7cad82740435646770f5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/68/04/ea/55bcff411d9f6eb0a064d2b1997dfd733640d8659ff2c98dbe
  Created wheel for inotify-simple: filename=inotify_simple-1.2.1-py3-none-any.whl size=8203 sha256=6aa6b709b2b172dab169f533f9442294a11b51ee925fbdf6212fc54d45a9eca3
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a5/34/f6/5d4eae9a326dc637344cd68353ddd119e9f6f575e925502888
Successfully built sagemaker-containers inotify-simple
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [94]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="./",
                    role=role,
                    framework_version='1.0.0',
                    py_version='py3',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path='s3://{}/{}/output'.format(sagemaker_session.default_bucket(), prefix),
                    hyperparameters={
                        'epochs': 150,
                        'learning_rate': 0.001
                    })

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [5]:
output_path='s3://{}/{}'.format(sagemaker_session.default_bucket(), prefix)
#estimator.fit({'train':s3://my-bucket/training_data})

In [6]:
print(output_path)

s3://sagemaker-us-east-2-769207522942/emotion-detection


In [90]:
upload_data = sagemaker_session.upload_data(path='./requirements.txt', bucket=bucket, key_prefix=prefix)

print('upload_data : {}'.format(upload_data))
#sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

upload_data : s3://sagemaker-us-east-2-769207522942/emotion-detection/requirements.txt


In [95]:
estimator.fit({'training':output_path})

2020-12-05 14:48:33 Starting - Starting the training job...
2020-12-05 14:48:35 Starting - Launching requested ML instances......
2020-12-05 14:49:54 Starting - Preparing the instances for training............
2020-12-05 14:51:56 Downloading - Downloading input data......
2020-12-05 14:52:50 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-05 14:53:16,645 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-05 14:53:16,672 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-05 14:53:19,691 sagemaker_pytorch_container.training INFO     Invoking user training script.

2020-12-05 14:53:15 Training - Training image download completed. Training in progress.2020-12-05 14:53:31,373 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-12-05 14:53:31,374 

  Running setup.py bdist_wheel for train: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-xzg0hiyc/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
  Running setup.py bdist_wheel for inotify-simple: started
  Running setup.py bdist_wheel for inotify-simple: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/28/77/f9/52cc89b27110b3fe0df40290275bd1151db9d0c7b15733cc3b
  Running setup.py bdist_wheel for psutil: started
  Running setup.py bdist_wheel for psutil: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/42/32/da/8b12fd6b138c733efd03cfde6c6c8191a32842f9e82aa45fbf
Successfully built sagemaker-containers train inotify-simple psutil
  Found existing installation: Werkzeug 0.15.4
    Uninstalling Werkzeug-0.15.4:
      Successfully uninstalled Werkzeug-0.15.4
  Found existing installation: Flask 1.0.3
    Uninstalling Flask-1.0.3:
      Successfully uninstalled Flask-1.0.3
  Found existing i

Epoch: 16, NLLLoss: 1.5628374993801117
Epoch 17/150.. Train loss: 1.563.. Valid loss: 1.521.. Validation accuracy: 0.459
Epoch: 17, NLLLoss: 1.5653798222541808
Epoch 18/150.. Train loss: 1.565.. Valid loss: 1.476.. Validation accuracy: 0.450
Epoch: 18, NLLLoss: 1.5493200898170472
Epoch 19/150.. Train loss: 1.549.. Valid loss: 1.543.. Validation accuracy: 0.443
Epoch: 19, NLLLoss: 1.5219359517097473
Epoch 20/150.. Train loss: 1.522.. Valid loss: 1.463.. Validation accuracy: 0.454
Epoch: 20, NLLLoss: 1.501008892059326
Epoch 21/150.. Train loss: 1.501.. Valid loss: 1.440.. Validation accuracy: 0.475
Epoch: 21, NLLLoss: 1.481653732061386
Epoch 22/150.. Train loss: 1.482.. Valid loss: 1.445.. Validation accuracy: 0.471
Epoch: 22, NLLLoss: 1.455172097682953
Epoch 23/150.. Train loss: 1.455.. Valid loss: 1.388.. Validation accuracy: 0.473
Epoch: 23, NLLLoss: 1.446303015947342
Epoch 24/150.. Train loss: 1.446.. Valid loss: 1.375.. Validation accuracy: 0.487
Epoch: 24, NLLLoss: 1.47420535087585

Epoch: 76, NLLLoss: 0.6656875357031822
Epoch 77/150.. Train loss: 0.666.. Valid loss: 1.255.. Validation accuracy: 0.591
Epoch: 77, NLLLoss: 0.5946455493569374
Epoch 78/150.. Train loss: 0.595.. Valid loss: 1.288.. Validation accuracy: 0.584
Epoch: 78, NLLLoss: 0.6205313205718994
Epoch 79/150.. Train loss: 0.621.. Valid loss: 1.355.. Validation accuracy: 0.606
Epoch: 79, NLLLoss: 0.5732758566737175
Epoch 80/150.. Train loss: 0.573.. Valid loss: 1.367.. Validation accuracy: 0.613
Epoch: 80, NLLLoss: 0.6071443036198616
Epoch 81/150.. Train loss: 0.607.. Valid loss: 1.339.. Validation accuracy: 0.595
Epoch: 81, NLLLoss: 0.5833615556359291
Epoch 82/150.. Train loss: 0.583.. Valid loss: 1.342.. Validation accuracy: 0.611
Epoch: 82, NLLLoss: 0.5398026123642922
Epoch 83/150.. Train loss: 0.540.. Valid loss: 1.405.. Validation accuracy: 0.620
Epoch: 83, NLLLoss: 0.5382673189043998
Epoch 84/150.. Train loss: 0.538.. Valid loss: 1.361.. Validation accuracy: 0.603
Epoch: 84, NLLLoss: 0.5745655104

In [48]:
# Get the name of the training job, 
training_job_name='sagemaker-pytorch-2020-12-05-09-07-48-004'

# where the model is saved, by default
model_key = os.path.join(prefix, "output",training_job_name, 'output/model.tar.gz')
print(model_key)

#s3://sagemaker-us-east-2-769207522942/emotion-detection/output/sagemaker-pytorch-2020-12-05-09-07-48-004/output/model.tar.gz 
# download and unzip model
boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

emotion-detection/output/sagemaker-pytorch-2020-12-05-09-07-48-004/output/model.tar.gz


2304

In [49]:
!ls -l

total 93960
-rw-rw-r-- 1 ec2-user ec2-user       96 Dec  4 06:17 category.json
drwxrwxr-x 2 ec2-user ec2-user     4096 Dec  3 12:14 data
-rw-rw-r-- 1 ec2-user ec2-user   265566 Dec  5 10:16 FacialExpression.ipynb
drwxrwxr-x 2 ec2-user ec2-user   638976 Dec  3 12:14 images
-rw-rw-r-- 1 ec2-user ec2-user    11357 Dec  3 12:14 LICENSE
-rw-r--r-- 1 ec2-user ec2-user 52968650 Dec  5 09:23 model.pth
-rw-rw-r-- 1 ec2-user ec2-user     2974 Dec  5 07:33 model.py
-rw-rw-r-- 1 ec2-user ec2-user 41570529 Dec  5 10:15 model.tar.gz
-rw-rw-r-- 1 ec2-user ec2-user     2762 Dec  4 16:40 predict.py
drwxrwxr-x 2 ec2-user ec2-user     4096 Dec  4 12:58 __pycache__
drwxrwxr-x 2 ec2-user ec2-user     4096 Dec  3 12:14 python
drwxrwxr-x 2 ec2-user ec2-user     4096 Dec  3 12:14 R
-rw-rw-r-- 1 ec2-user ec2-user      963 Dec  3 12:14 README.md
-rw-rw-r-- 1 ec2-user ec2-user       21 Dec  4 09:56 requirements.txt
drwxrwxr-x 6 ec2-user ec2-user   618496 Dec  4 09:58 selectedImages
-rw-rw-r-- 1 ec2-user ec2-user

In [81]:
#from sagemaker.pytorch import PyTorchModel
#model = PyTorchModel(model_data=estimator.model_data,
#                     role = role,
#                     framework_version='1.0.0',
#                     py_version='py3')
"""model = EmotionClassifier()

    # Load the store model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(f))

    model.to(device).eval()"""
model_dir = "./"
model_path = os.path.join(model_dir, 'model.pth')
with open(model_path, 'rb') as f:
    model.load_state_dict(torch.load(f))
#model =model.load_state_dict(torch.load())
model.to(device)
test_loss = 0
accuracy = 0
with torch.no_grad():
    model.eval()
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        m = nn.LogSoftmax(dim=1)
        batch_loss = loss_fn(m(logps), labels)

        test_loss += batch_loss.item()
        #accuracy = 0
        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(testloader):.3f}.. "
	f"Test accuracy: {accuracy/len(testloader):.3f}")

Test loss: 12953457619107840.000.. Test accuracy: 0.116


In [52]:
print(model)

EmotionClassifier(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.25, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.25, inplace=False)

In [1]:
import torchvision.models as models

In [2]:
vgg16 = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ec2-user/.cache/torch/checkpoints/vgg16-397923af.pth


In [3]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
import torch.nn as nn
vgg16.classifier[6] = nn.Linear(4096,7)

In [22]:
model = models.vgg16()
model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                         nn.ReLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(4096, 1024),
                                         nn.ReLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(1024,512),
                                         nn.ReLU(),
                                         nn.Dropout(0.2),
                                         nn.Linear(512, 7))
model_dir = "./"
model_path = os.path.join(model_dir, 'modelVGG.pth')
with open(model_path, 'rb') as f:
    model.load_state_dict(torch.load(f))
#model =model.load_state_dict(torch.load())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = vgg16
model.to(device)
test_loss = 0
accuracy = 0
with torch.no_grad():
    model.eval()
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        m = nn.LogSoftmax(dim=1)
        batch_loss = loss_fn(m(logps), labels)

        test_loss += batch_loss.item()
        #accuracy = 0
        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        print(f"Test loss: {test_loss/len(testloader):.3f}.. "
                f"Test accuracy: {accuracy/len(testloader):.3f}")
print(f"Test loss: {test_loss/len(testloader):.3f}.. "
	f"Test accuracy: {accuracy/len(testloader):.3f}")

Test loss: 0.050.. Test accuracy: 0.000
Test loss: 0.099.. Test accuracy: 0.000
Test loss: 0.149.. Test accuracy: 0.000
Test loss: 0.199.. Test accuracy: 0.000
Test loss: 0.249.. Test accuracy: 0.000
Test loss: 0.300.. Test accuracy: 0.000
Test loss: 0.353.. Test accuracy: 0.000
Test loss: 0.405.. Test accuracy: 0.000
Test loss: 0.458.. Test accuracy: 0.000
Test loss: 0.533.. Test accuracy: 0.000
Test loss: 0.573.. Test accuracy: 0.000
Test loss: 0.607.. Test accuracy: 0.000
Test loss: 0.641.. Test accuracy: 0.000
Test loss: 0.675.. Test accuracy: 0.000
Test loss: 0.709.. Test accuracy: 0.000
Test loss: 0.742.. Test accuracy: 0.000
Test loss: 0.776.. Test accuracy: 0.000
Test loss: 0.810.. Test accuracy: 0.000
Test loss: 0.844.. Test accuracy: 0.000
Test loss: 0.878.. Test accuracy: 0.000
Test loss: 0.911.. Test accuracy: 0.023
Test loss: 0.945.. Test accuracy: 0.047
Test loss: 0.979.. Test accuracy: 0.070
Test loss: 1.012.. Test accuracy: 0.093
Test loss: 1.046.. Test accuracy: 0.116


In [7]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train-VGG16.py",
                    source_dir="./",
                    role=role,
                    framework_version='1.0.0',
                    py_version='py3',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path='s3://{}/{}/output'.format(sagemaker_session.default_bucket(), prefix),
                    hyperparameters={
                        'epochs': 50,
                        'learning_rate': 0.001
                    })

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
estimator.fit({'training':output_path})

2020-12-10 05:33:21 Starting - Starting the training job...
2020-12-10 05:33:44 Starting - Launching requested ML instancesProfilerReport-1607578374: InProgress
......
2020-12-10 05:34:45 Starting - Preparing the instances for training.........
2020-12-10 05:36:06 Downloading - Downloading input data...
2020-12-10 05:36:46 Training - Downloading the training image...
2020-12-10 05:37:21 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-10 05:37:22,980 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-10 05:37:23,006 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-10 05:37:23,007 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-10 05:37:38,495 sagemaker-containers INFO     Module train-VGG16 does not provide a setup.py. 
Gene

  Running setup.py bdist_wheel for train-VGG16: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgyxailu/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
  Running setup.py bdist_wheel for inotify-simple: started
  Running setup.py bdist_wheel for inotify-simple: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/28/77/f9/52cc89b27110b3fe0df40290275bd1151db9d0c7b15733cc3b
  Running setup.py bdist_wheel for psutil: started
  Running setup.py bdist_wheel for psutil: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/42/32/da/8b12fd6b138c733efd03cfde6c6c8191a32842f9e82aa45fbf
Successfully built sagemaker-containers train-VGG16 inotify-simple psutil
  Found existing installation: Werkzeug 0.15.4
    Uninstalling Werkzeug-0.15.4:
      Successfully uninstalled Werkzeug-0.15.4
  Found existing installation: Flask 1.0.3
    Uninstalling Flask-1.0.3:
      Successfully uninstalled Flask-1.0.3
  Foun

Epoch: 16, NLLLoss: 1.617160427570343
Epoch 17/50.. Train loss: 1.617.. Valid loss: 1.586.. Validation accuracy: 0.321
Epoch: 17, NLLLoss: 1.6096008479595185
Epoch 18/50.. Train loss: 1.610.. Valid loss: 1.504.. Validation accuracy: 0.328
Epoch: 18, NLLLoss: 1.5825287580490113
Epoch 19/50.. Train loss: 1.583.. Valid loss: 1.496.. Validation accuracy: 0.321
Epoch: 19, NLLLoss: 1.579994022846222
Epoch 20/50.. Train loss: 1.580.. Valid loss: 1.506.. Validation accuracy: 0.324
Epoch: 20, NLLLoss: 1.5792052805423737
Epoch 21/50.. Train loss: 1.579.. Valid loss: 1.496.. Validation accuracy: 0.319
Epoch: 21, NLLLoss: 1.5815246641635894
Epoch 22/50.. Train loss: 1.582.. Valid loss: 1.532.. Validation accuracy: 0.326
Epoch: 22, NLLLoss: 1.5947966992855072
Epoch 23/50.. Train loss: 1.595.. Valid loss: 1.498.. Validation accuracy: 0.324
Epoch: 23, NLLLoss: 1.567913830280304
Epoch 24/50.. Train loss: 1.568.. Valid loss: 1.499.. Validation accuracy: 0.321
Epoch: 24, NLLLoss: 1.6401480555534362
Epoc

In [11]:
#sagemaker-pytorch-2020-12-10-05-32-54-649
import os
training_job_name='sagemaker-pytorch-2020-12-10-05-32-54-649'

# where the model is saved, by default
model_key = os.path.join(prefix, "output",training_job_name, 'output/model.tar.gz')
print(model_key)

#s3://sagemaker-us-east-2-769207522942/emotion-detection/output/sagemaker-pytorch-2020-12-05-09-07-48-004/output/model.tar.gz 
# download and unzip model
boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')

emotion-detection/output/sagemaker-pytorch-2020-12-10-05-32-54-649/output/model.tar.gz


0

In [13]:
!ls -l

total 921876
-rw-rw-r-- 1 ec2-user ec2-user        96 Dec  4 06:17 category.json
drwxrwxr-x 2 ec2-user ec2-user      4096 Dec  3 12:14 data
-rw-rw-r-- 1 ec2-user ec2-user    325204 Dec 10 06:28 FacialExpression.ipynb
drwxrwxr-x 2 ec2-user ec2-user    638976 Dec  3 12:14 images
-rw-rw-r-- 1 ec2-user ec2-user     11357 Dec  3 12:14 LICENSE
-rw-rw-r-- 1 ec2-user ec2-user      3014 Dec  5 12:10 model.py
-rw-rw-r-- 1 ec2-user ec2-user 453431005 Dec 10 06:29 model.tar.gz
-rw-r--r-- 1 ec2-user ec2-user 488817546 Dec 10 06:23 modelVGG.pth
-rw-rw-r-- 1 ec2-user ec2-user      2762 Dec  4 16:40 predict.py
drwxrwxr-x 2 ec2-user ec2-user      4096 Dec  5 12:06 __pycache__
drwxrwxr-x 2 ec2-user ec2-user      4096 Dec  3 12:14 python
drwxrwxr-x 2 ec2-user ec2-user      4096 Dec  3 12:14 R
-rw-rw-r-- 1 ec2-user ec2-user       963 Dec  3 12:14 README.md
-rw-rw-r-- 1 ec2-user ec2-user        20 Dec  5 12:33 requirements.txt
drwxrwxr-x 6 ec2-user ec2-user    618496 Dec  4 09:58 selectedImages
-rw-rw-r-- 